In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from flask import Flask, jsonify
import threading

app = Flask(__name__)


# Define the preprocessing function for TensorFlow (same as during training)
def preprocess_image(image):
    #img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert BGR to RGB For Efficient Net + Custom
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB (3 channels) For Dense Net
    img = cv2.resize(img, (48, 48))  # Resize to model's expected input size
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)
    img = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
    return img


@app.route("/data", methods=["GET"])
def send_data():
    global previous_emotion, current_emotion
    if current_emotion != previous_emotion:
        previous_emotion = current_emotion
        output_data = {"Emotion: ": current_emotion}        
        return jsonify(output_data)
    else:
        return jsonify(output_data)

        
def run_flask_app():
    app.run(debug=False, use_reloader=False, port=5000)
    
# Load the trained TensorFlow model
#model = load_model('emotion_detection_model.h5')
#model = load_model('efficientnet_model.h5')
model = load_model('densenet_model.h5')

current_emotion = "Neutral"
previous_emotion = "Neutral"
# Starting the Flask server in a separate thread
thread = threading.Thread(target=run_flask_app)
thread.start()

# Start webcam capture
video_capture = cv2.VideoCapture(0)

# Emotion labels (adjust to match the labels your model uses)
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", 'Neutral','Sad','Surprise']

while True:
    ret, frame = video_capture.read()  # Capture a frame from the webcam
    
    # Preprocess the frame for the model
    img = preprocess_image(frame)

    # Runs model to get predictions
    predictions = model.predict(img, verbose=0)
    predicted_class = np.argmax(predictions, axis=1)[0]

    predicted_emotion = emotion_labels[predicted_class]
    current_emotion = predicted_emotion

    # Display the predicted emotion on the frame
    cv2.putText(frame, f'Emotion: {predicted_emotion}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show the frame with emotion label
    cv2.imshow('Emotion Recognition', frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
video_capture.release()
cv2.destroyAllWindows()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Dec/2024 21:23:22] "GET /data HTTP/1.1" 200 -
ERROR:__main__:Exception on /data [GET]
Traceback (most recent call last):
  File "C:\Users\viole\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\viole\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\viole\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\viole\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(